# Experiment 01

First Tests from the predictions.

In [1]:
import numpy as np
import pandas as pd
import math

import sys
import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc

import warnings
warnings.filterwarnings('ignore')


def getPeriod2(df, begin, end, resetIndex = False):
    """
    Returns the df in the chosen interval
    
    Object begin: Start date forrmated as 'yyyy.mm.dd'.
    Object   end: End date 'yyyy.mm.dd'.

    returns a dataframe with the historic of the selected period
    """
    
    indexBegin = df[df['date']==begin].index[0]
    indexEnd = df[df['date']==end].tail(1).index[0]
    
    if (resetIndex):
        return df[(df.index >= indexBegin) & (df.index <= indexEnd)].reset_index(drop=True)
    else: 
        return df[(df.index >= indexBegin) & (df.index <= indexEnd)]

In [2]:
dfColumns = ['date','time','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']

TEST_BEGIN = '2021.04.06'
TEST_END = '2021.06.01'

TRAIN_BEGIN = '2020.03.31'
TRAIN_END = '2021.05.31'

TIMES = 10

In [3]:
df = pd.read_csv('../Data/PETR4_M15.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

df2 = pd.read_csv('../Data/VALE3_M15.csv', sep = "\t")
df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

initialDf_PETR4 = getPeriod2(df, TEST_BEGIN, TEST_END)
initialDf_VALE3 = getPeriod2(df2, TEST_BEGIN, TEST_END)

In [5]:
initialDf_PETR4

,date,time,open,high,low,close
6197,2021.04.06,10:00:00,23.39,23.47,23.32,23.32
6198,2021.04.06,10:15:00,23.32,23.38,23.30,23.36
6199,2021.04.06,10:30:00,23.36,23.53,23.33,23.52
6200,2021.04.06,10:45:00,23.52,23.58,23.39,23.48
6201,2021.04.06,11:00:00,23.48,23.49,23.36,23.39
...,...,...,...,...,...,...
7311,2021.06.01,15:45:00,27.14,27.14,27.05,27.06
7312,2021.06.01,16:00:00,27.06,27.09,27.01,27.02
7313,2021.06.01,16:15:00,27.01,27.17,27.01,27.14
7314,2021.06.01,16:30:00,27.15,27.31,27.11,27.27


In [6]:
initialDf_VALE3

,date,time,open,high,low,close
6186,2021.04.06,10:00:00,103.80,104.35,103.52,103.77
6187,2021.04.06,10:15:00,103.77,103.80,102.92,103.23
6188,2021.04.06,10:30:00,103.23,104.25,103.00,104.23
6189,2021.04.06,10:45:00,104.23,104.56,103.70,104.03
6190,2021.04.06,11:00:00,104.03,104.12,103.15,103.34
...,...,...,...,...,...,...
7301,2021.06.01,15:45:00,113.84,113.95,113.61,113.72
7302,2021.06.01,16:00:00,113.72,114.00,113.70,113.99
7303,2021.06.01,16:15:00,113.99,114.00,113.90,113.96
7304,2021.06.01,16:30:00,113.97,114.00,113.88,113.88


In [28]:
pred_PETR4 = pd.read_pickle('../Data/PETR4_Pred_with_confidence.pkl')
pred_PETR4.head(5)

,high,low,high_pred,low_pred,uncertainty,date
1299,23.58,23.13,23.624167,23.038600,0.264795,2021.04.06
1300,23.55,23.15,23.563533,23.027933,0.247948,2021.04.07
1301,23.26,22.61,23.551200,23.009967,0.225871,2021.04.08
1302,23.01,22.61,23.154067,22.493967,0.318393,2021.04.09
1303,23.58,23.01,23.675200,22.946867,0.401854,2021.04.12


In [29]:
pred_VALE3 = pd.read_pickle('../Data/VALE3_Pred_with_confidence.pkl')
pred_VALE3.head(15)

,high,low,high_pred,low_pred,uncertainty,date
1299,104.56,102.05,102.036967,98.697267,2.018520,2021.04.06
1300,105.32,102.15,102.133233,98.759433,2.090933,2021.04.07
1301,105.30,104.12,102.025533,98.788033,1.981552,2021.04.08
1302,104.66,102.94,102.055500,98.819167,1.938339,2021.04.09
1303,103.63,102.21,102.029900,98.800400,1.984671,2021.04.12
1304,104.29,102.85,102.029900,98.800400,1.984671,2021.04.13
1305,107.78,103.77,102.029900,98.800400,1.984671,2021.04.14
1306,109.88,107.55,102.029900,98.800400,1.984671,2021.04.15
1307,109.11,107.50,102.029900,98.800400,1.984671,2021.04.16
1308,109.88,107.64,102.029900,98.800400,1.984671,2021.04.19


### Início do sistema

In [30]:
def getDay(df, day):
    return df[df['date']==day]

def buy(value, availableMoney, opr='normal'):
    lotValue = value*100
    quantity = math.floor(availableMoney/lotValue)
    
    amount = quantity*lotValue
    remainingMoney = availableMoney - amount
    
    return remainingMoney, quantity

def sell(value, amount, opr='normal'):
    if(opr == 'normal'):
        return value*amount*100
    else:
        return value*amount*100

In [31]:
dayDf = getPeriod2(initialDf_PETR4, '2021.04.06', '2021.04.06')
dayDf.head(5)

,date,time,open,high,low,close
6197,2021.04.06,10:00:00,23.39,23.47,23.32,23.32
6198,2021.04.06,10:15:00,23.32,23.38,23.30,23.36
6199,2021.04.06,10:30:00,23.36,23.53,23.33,23.52
6200,2021.04.06,10:45:00,23.52,23.58,23.39,23.48
6201,2021.04.06,11:00:00,23.48,23.49,23.36,23.39


In [71]:
def runStrategy(df, dailyDf, Money, stopLoss, std, uncertainty_threshold):
    size = len(df.index)

    Money = Money
    stopLoss = stopLoss

    orders = []

    for day in range(0,size):
        currentDay =  df.iloc[day].date

        lowPred = df.iloc[day].low_pred*(1+std)
        highPred = df.iloc[day].high_pred*(1-std)
        day_uncertainty = df.iloc[day].uncertainty
        if day_uncertainty > uncertainty_threshold:
            continue

        dayDf = getPeriod2(dailyDf, currentDay, currentDay)
        daySize = len(dayDf.index)

        op = 'none'
        quantity = 0
        trade = []

        for i in range(0,daySize):
            currentTime = dayDf.iloc[i]

            if((op == 'buy') and (currentTime.close < opPrice*(1-stopLoss) or currentTime.close >= highPred or i == daySize-1)):
                isStop = False
                if(currentTime.close < opPrice*(1-stopLoss)): isStop = True
                aux = sell(currentTime.close, quantity)
                Money = Money + aux 
                trade = trade + [currentTime.time, 'Sell', currentTime.close, currentTime.close-opPrice,(currentTime.close-opPrice)*100*quantity, isStop, Money, day_uncertainty]
                orders.append(trade)

                quantity = 0
                op = 'none'

            elif((op == 'sell') and (currentTime.close > opPrice*(1+stopLoss) or currentTime.close <= lowPred or i == daySize-1)):
                isStop = False
                if(currentTime.close > opPrice*(1+stopLoss)): isStop = True

                aux = sell(currentTime.close, quantity)
                Money = Money + (opPrice*100*quantity) + ((opPrice*100*quantity) - aux) 
                trade = trade + [currentTime.time, 'Buy', currentTime.close, opPrice-currentTime.close,(opPrice-currentTime.close)*100*quantity, isStop, Money, day_uncertainty]
                orders.append(trade)

                quantity = 0
                op = 'none'

            elif(op == 'none' and i < daySize-1):
                if(currentTime.close <= lowPred):
                    Money, quantity = buy(currentTime.close, Money)
                    opPrice = currentTime.close
                    op = 'buy'

                    trade = [currentTime.date, quantity, currentTime.time, 'Buy', currentTime.close]

                elif(currentTime.close >=  highPred):
                    Money, quantity = buy(currentTime.close, Money)
                    opPrice = currentTime.close
                    op = 'sell'

                    trade = [currentTime.date, quantity, currentTime.time, 'Sell', currentTime.close]
        print('eday '+str(day)+' money: '+str(Money))
    
    ordersDf = pd.DataFrame(orders)
    ordersDf.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount', 'uncertainty']
    tradedDays = ordersDf['date'].nunique()
    return ordersDf, tradedDays

In [72]:
# testPetr = runStrategy(pred_PETR4, initialDf_PETR4, 500000, 0.01, 0.01, 0.34)
# print('Accuracy: '+str(round(100*len(testPetr[testPetr['profit'] > 0].index)/len(testPetr.index), 2)))

resultados = []
for i in np.arange(0.2, 0.51, 0.02):
    for j in np.arange(0.00, 0.03, 0.01):
        testPetr, tradedDays = runStrategy(pred_PETR4, initialDf_PETR4, 500000, j, 0.01, i)
        finalAmount = testPetr.tail(1)['Amount'].iloc[0]
        acuracia = round(100*len(testPetr[testPetr['profit'] > 0].index)/len(testPetr.index), 2)

        resultados.append([i, j, finalAmount, acuracia, tradedDays])

# testVale = runStrategy(pred_VALE3, initialDf_VALE3, 100000, 0.01, 0.02, 1.848)
# print('Accuracy: '+str(round(100*len(testVale[testVale['profit'] > 0].index)/len(testVale.index), 2)))

eday 18 money: 498104.00000000006
eday 29 money: 504614.00000000006
eday 36 money: 509061.0000000001
eday 39 money: 514646.0
eday 18 money: 495779.99999999994
eday 29 money: 502255.0
eday 36 money: 506893.0000000001
eday 39 money: 512650.00000000006
eday 18 money: 494513.99999999994
eday 29 money: 500989.0
eday 36 money: 505609.0000000002
eday 39 money: 511351.00000000006
eday 12 money: 499782.00000000006
eday 13 money: 496433.9999999999
eday 14 money: 493972.9999999999
eday 17 money: 502573.99999999977
eday 18 money: 500668.99999999994
eday 21 money: 505263.00000000006
eday 29 money: 511878.00000000006
eday 31 money: 516377.00000000023
eday 35 money: 516379.0000000001
eday 36 money: 520909.0000000001
eday 39 money: 526623.0000000001
eday 12 money: 499790.00000000006
eday 13 money: 498954.0000000001
eday 14 money: 492459.0000000001
eday 17 money: 501266.0000000001
eday 18 money: 497054.0000000001
eday 21 money: 501620.0000000001
eday 29 money: 508200.0000000001
eday 31 money: 510682.00

In [73]:
bestUncertainty = pd.DataFrame(resultados,columns=['uncertainty','std','finalAmount','accuracy','tradedDays'])
bestUncertainty = bestUncertainty.sort_values(by='accuracy', ascending=False)
print(bestUncertainty.head(15))

    uncertainty   std  finalAmount  accuracy  tradedDays
1          0.20  0.01     512650.0     66.67           4
11         0.26  0.02     517439.0     65.52          20
2          0.20  0.02     511351.0     63.64           4
10         0.26  0.01     524105.0     62.69          20
4          0.22  0.01     525686.0     61.76          11
14         0.28  0.02     522295.0     61.76          27
20         0.32  0.02     515706.0     61.54          34
5          0.22  0.02     523044.0     61.29          11
17         0.30  0.02     503611.0     60.00          31
23         0.34  0.02     504054.0     60.00          35
19         0.32  0.01     523802.0     60.00          34
13         0.28  0.01     528624.0     58.97          27
22         0.34  0.01     512430.0     58.70          35
16         0.30  0.01     511444.0     58.62          31
35         0.42  0.02     510710.0     58.24          40


In [45]:
testPetr.tail(10)

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount,uncertainty
82,2021.05.27,39,16:00:00,Buy,25.98,16:15:00,Sell,26.01,0.03,117.0,False,103460.0,0.190196
83,2021.05.27,39,16:30:00,Buy,25.99,16:45:00,Sell,25.91,-0.08,-312.0,False,103148.0,0.190196
84,2021.05.28,39,10:00:00,Sell,26.41,11:00:00,Buy,26.82,-0.41,-1599.0,True,101549.0,0.335756
85,2021.05.28,37,11:15:00,Sell,26.82,16:45:00,Buy,26.99,-0.17,-629.0,False,100920.0,0.335756
86,2021.05.31,37,10:15:00,Buy,26.92,10:30:00,Sell,27.00,0.08,296.0,False,101216.0,0.281216
87,2021.05.31,37,12:00:00,Buy,26.85,16:45:00,Sell,26.87,0.02,74.0,False,101290.0,0.281216
88,2021.06.01,37,10:00:00,Buy,27.37,10:15:00,Sell,27.52,0.15,555.0,False,101845.0,0.195980
89,2021.06.01,37,10:30:00,Buy,27.36,10:45:00,Sell,27.36,0.00,0.0,False,101845.0,0.195980
90,2021.06.01,37,11:00:00,Sell,27.42,11:30:00,Buy,27.37,0.05,185.0,False,102030.0,0.195980
91,2021.06.01,37,11:45:00,Buy,27.18,16:45:00,Sell,27.29,0.11,407.0,False,102437.0,0.195980


In [11]:
testVale.tail(10)

NameError: name 'testVale' is not defined

In [12]:
df = pred_VALE3
dailyDf = initialDf_VALE3
size = len(df.index)

Money = 100000
stopLoss = 0.01
LastTrade = ['None',10,100]

orders = []

for day in range(0,size):
    currentDay =  df.iloc[day].date

    lowPred = df.iloc[day].low_pred*1.01
    highPred = df.iloc[day].high_pred*0.99
    
    dayDf = getPeriod2(dailyDf, currentDay, currentDay)
    daySize = len(dayDf.index)
    
    op = 'none'
    quantity = 0
    trade = []
    
    print('Begining day '+str(day)+' money: '+str(Money))
    for i in range(0,daySize):
        currentTime = dayDf.iloc[i]
        
        if((op == 'buy') and (currentTime.close < opPrice*(1-stopLoss) or currentTime.close >= highPred or i == daySize-1)):
            isStop = False
            if(currentTime.close < opPrice*(1-stopLoss)): isStop = True
            aux = sell(currentTime.close, quantity)
            Money = Money + aux 
            trade = trade + [currentTime.time, 'Sell', currentTime.close, currentTime.close-opPrice,(currentTime.close-opPrice)*100*quantity, isStop, Money]
            orders.append(trade)
  
            quantity = 0
            op = 'none'
        
        elif((op == 'sell') and (currentTime.close > opPrice*(1+stopLoss) or currentTime.close <= lowPred or i == daySize-1)):
            isStop = False
            if(currentTime.close > opPrice*(1+stopLoss)): isStop = True

            aux = sell(currentTime.close, quantity)
            Money = Money + (opPrice*100*quantity) + ((opPrice*100*quantity) - aux) 
            trade = trade + [currentTime.time, 'Buy', currentTime.close, opPrice-currentTime.close,(opPrice-currentTime.close)*100*quantity, isStop, Money]
            orders.append(trade)
            
            quantity = 0
            op = 'none'
        
        elif(op == 'none' and i < daySize-1 ):
            if(currentTime.close <= lowPred):
                Money, quantity = buy(currentTime.close, Money)
                opPrice = currentTime.close
                op = 'buy'
                
                trade = [currentTime.date, quantity, currentTime.time, 'Buy', currentTime.close]
                
            elif(currentTime.close >=  highPred):
                Money, quantity = buy(currentTime.close, Money)
                opPrice = currentTime.close
                op = 'sell'
                
                trade = [currentTime.date, quantity, currentTime.time, 'Sell', currentTime.close]

test2 = pd.DataFrame(orders)
test2.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']
test2.tail(60)

test2 = pd.DataFrame(orders)
test2.columns = ['date','batches','time_op1','op1','value_op1','time_op2','op2','value_op2', 'profit', 'totalProfit', 'stop','Amount']

print('Accuracy: '+str(round(100*len(test2[test2['profit'] > 0].index)/len(test2.index), 2)))
test2.head(10)

Begining day 0 money: 100000
Begining day 1 money: 101548.0
Begining day 2 money: 100225.0
Begining day 3 money: 100864.0
Begining day 4 money: 101611.0
Begining day 5 money: 101053.0
Begining day 6 money: 101494.0
Begining day 7 money: 99325.0
Begining day 8 money: 100459.0
Begining day 9 money: 100261.0
Begining day 10 money: 101944.0
Begining day 11 money: 103375.0
Begining day 12 money: 103888.0
Begining day 13 money: 103987.0
Begining day 14 money: 104860.00000000001
Begining day 15 money: 103411.00000000001
Begining day 16 money: 101494.00000000001
Begining day 17 money: 102214.00000000001
Begining day 18 money: 102628.00000000001
Begining day 19 money: 103726.00000000001
Begining day 20 money: 103330.00000000001
Begining day 21 money: 104734.00000000001
Begining day 22 money: 102601.00000000001
Begining day 23 money: 103345.00000000001
Begining day 24 money: 107089.00000000001
Begining day 25 money: 103568.00000000004
Begining day 26 money: 106560.00000000004
Begining day 27 mon

,date,batches,time_op1,op1,value_op1,time_op2,op2,value_op2,profit,totalProfit,stop,Amount
0,2021.04.06,9,10:00:00,Sell,103.77,16:45:00,Buy,102.05,1.72,1548.0,False,101548.0
1,2021.04.07,9,10:00:00,Sell,102.76,13:30:00,Buy,103.95,-1.19,-1071.0,True,100477.0
2,2021.04.07,9,13:45:00,Sell,104.28,16:45:00,Buy,104.56,-0.28,-252.0,False,100225.0
3,2021.04.08,9,10:00:00,Sell,105.21,16:45:00,Buy,104.50,0.71,639.0,False,100864.0
4,2021.04.09,9,10:00:00,Sell,103.83,16:45:00,Buy,103.00,0.83,747.0,False,101611.0
5,2021.04.12,9,10:00:00,Sell,102.78,16:45:00,Buy,103.40,-0.62,-558.0,False,101053.0
6,2021.04.13,9,10:00:00,Sell,104.07,16:45:00,Buy,103.58,0.49,441.0,False,101494.0
7,2021.04.14,9,10:00:00,Sell,104.18,10:45:00,Buy,105.61,-1.43,-1287.0,True,100207.0
8,2021.04.14,9,11:00:00,Sell,106.00,11:30:00,Buy,107.44,-1.44,-1296.0,True,98911.0
9,2021.04.14,9,11:45:00,Sell,107.46,16:45:00,Buy,107.00,0.46,414.0,False,99325.0
